In [1]:
import emip_toolkit as emtk
import filter_fixation as ff
import numpy as np
import pandas as pd
import json
import time
import copy

%matplotlib tk      
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%load_ext autoreload
%autoreload 2

1. draw() method draws the fixation and saccades of given trial, it skips deleted fixations
2. is_fixation() method finds whether the location user clicks belongs to a fixation
3. find_index() method finds index of modified fixation
4. changed_fix() method counts the number of modified fixations.

Ashley, you can message me for any question on WeChat.

In [2]:
def draw(centers):
    fixs = []
    for data in centers.values():
        if data[0] == -1 and data[1] == -1: continue
        elif data[0] == -2 and data[1] == -2: continue
        else:
            x, y, duration = data
            fixs.append([x, y])
    fixs = np.array(fixs)
    
    l = plt.plot(fixs[:, 0], fixs[:, 1], alpha=0.4, c='orange')
    line = l.pop(0)
    
    scatter_color = ['yellow'] * len(fixs)
    if list(centers.values())[0][0] != -2: scatter_color[0] = 'red'
    scatter = plt.scatter(fixs[:, 0], fixs[:, 1], alpha=0.5, s=100, c=scatter_color)
    return line, scatter

def is_fixation(x, y, centers):
    if find_index(centers):
        # Empty space exists, new fixation needs to be drawn
        return None
    for index, (c_x, c_y, duration) in centers.items():
        # If click is on one fixation -> Hide it
        if c_x-12 < x < c_x+12 and c_y-12 < y < c_y+12:
            return index
    return None

def find_index(centers):
    for index, data in centers.items():
        if data[0] == -1 and data[1] == -1:
            return index
    return None

def changed_fix(participant_id):
    count = 0
    original = f'originals/R8S1_{participant_id}_ORIGINAL.json'
    with open(original, 'r') as file:
        original = json.loads(file.read())
        
    corrected = f'corrections/R8S1_{participant_id}_CORRECTED.json'
    with open(corrected, 'r') as file:
        corrected = json.loads(file.read())
    for key in original:
        if original[key] != corrected[key]:
            count += 1
    return count, len(original)

Modfiy the following cell before the edition to move to a new trial

In [3]:
path = 'datasets/GazeBase_TEX/Data/S_8331_S1_TEX.csv'

Run the following to do manual correction, a new window will pop.

1. Left click on one fixation to hide it, then left click again to modify its location.
2. Right click on one fixation to delete it.
3. Press 'Z' to undo.
4. Close the window by clicking the red close button to save the correction and count the statistics.

In [4]:
start = time.time()
fig = plt.figure(figsize=(12,8))


pixel_data = ff.process_GazeBase(path)
fixations, saccades = ff.filter_fixations(pixel_data)

path = path.split('/')[-1]
trial_id = int(path[8])
round_id = path[2]
participant_id = path[3:6]
imagepath = f"datasets/GazeBase_TEX/StimuliExamples/TEXScreenshots/TEX_R{round_id}S{trial_id}_bg.png"

img = mpimg.imread(imagepath)

# Initialize fixation centers
centers = {}
for index, fixation in fixations.items():
    fixation = fixation.get_fixation()
    duration = fixation[3]
    if duration < 100:
        continue
    x = fixation[4]
    y = fixation[5]
    centers[index] = [x, y, duration]

line, scatter = draw(centers)
history = []

def onclick(event):
    global line, scatter, centers, clicked, history

    if event.button == 1:
        x, y = event.xdata, event.ydata
        index = is_fixation(x, y, centers)

        if index:
            # If click is on one fixation -> Hide it
            history.append(copy.deepcopy(centers))
            centers[index][0] = -1
            centers[index][1] = -1
        else:
            # If click is on empty spot -> Find empty spot and fill with current position
            index = find_index(centers)
            if index:
                centers[index][0] = x
                centers[index][1] = y

        line.remove()
        scatter.remove()

        line, scatter = draw(centers)
        fig.canvas.draw()

        with open(f'corrections/R{round_id}S{trial_id}_{participant_id}_CORRECTED.json', 'w') as file:
            file.write(json.dumps(centers))
    elif event.button == 3:
        x, y = event.xdata, event.ydata
        index = is_fixation(x, y, centers)
        
        if index:
            history.append(copy.deepcopy(centers))
            centers[index][0] = -2
            centers[index][1] = -2
            
        line.remove()
        scatter.remove()

        line, scatter = draw(centers)
        fig.canvas.draw()

        with open(f'corrections/R{round_id}S{trial_id}_{participant_id}_CORRECTED.json', 'w') as file:
            file.write(json.dumps(centers))

    
def onpress(event):
    global history, line, scatter, centers

    if event.key == 'z':
        if len(history) > 0:
            centers = history.pop()

            line.remove()
            scatter.remove()

            line, scatter = draw(centers)
            fig.canvas.draw()

            with open(f'corrections/R{round_id}S{trial_id}_{participant_id}_CORRECTED.json', 'w') as file:
                file.write(json.dumps(centers))

def onclose(event):
    global participant_id
    end = time.time()
    total_time = round(end-start, 2)
    
    changed_fixations, total_fixations = changed_fix(participant_id)
    
    with open(f'corrections/R{round_id}S{trial_id}_{participant_id}_CORRECTED.json', 'r') as file:
        fixations_data = json.loads(file.read())
    
    output = {}
    output['total_time'] = total_time
    output['changed_fixations'] = changed_fixations
    output['total_fixations'] = total_fixations
    output['fixations_data'] = fixations_data
    
    with open(f'corrections/R{round_id}S{trial_id}_{participant_id}_CORRECTED.json', 'w') as file:
        file.write(json.dumps(output))

cid = fig.canvas.mpl_connect('button_press_event', onclick)
cid2 = fig.canvas.mpl_connect('close_event', onclose)
cid3 = fig.canvas.mpl_connect('key_press_event', onpress)
imgplot = plt.imshow(img)
plt.show()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/tkinter/__init__.py", line 1883, in __call__
    return self.func(*args)
  File "/Users/speng/Library/Python/3.8/lib/python/site-packages/matplotlib/backends/_backend_tk.py", line 219, in filter_destroy
    self.close_event()
  File "/Users/speng/Library/Python/3.8/lib/python/site-packages/matplotlib/backend_bases.py", line 1777, in close_event
    self.callbacks.process(s, event)
  File "/Users/speng/Library/Python/3.8/lib/python/site-packages/matplotlib/cbook/__init__.py", line 229, in process
    self.exception_handler(exc)
  File "/Users/speng/Library/Python/3.8/lib/python/site-packages/matplotlib/cbook/__init__.py", line 81, in _exception_printer
    raise exc
  File "/Users/speng/Library/Python/3.8/lib/python/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  F

1. First fixation in entire trial red
2. Undo button
3. Many undo